In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# list of the months
months=[ "january", "february", "march", "april",
    "may", "june", "july", "august",
    "september", "october", "november", "december"
]

# list of the remaining counties
counties = ['kisii','kisumu','kitui','kwale','laikipia', 'lamu','machakos',
            'makueni','mandera','marsabit', 'meru','migori','mombasa','murang-a','nairobi-city','nakuru'] 

# base url
base_url = "https://weatherandclimate.com"

# intialize list of all years
all_years=[]

# iterate upto year 2020
current_year=2010
for i in range(10):
    current_year+=1
    all_years.append(current_year)

# intialize and iterate through the counties,years and months
# create a list of all the urls
all_url=[]
for county in counties:
    for month in months:
        for year in all_years:
            url = f"{base_url}/{county}/{month}{-year}"
            all_url.append(url)
            

In [23]:
all_data=pd.DataFrame()
all_headers=[]



for url in all_url:
    response=requests.get(url)
    data=response.text
    soup=BeautifulSoup(data, "html.parser")
    table=soup.find('table',class_='tb8')

    # Extract headers 
    headers=table.find('tr')
    header=[hd.get_text(strip=True) for hd in headers.find_all('th')]
    all_headers.append(header)


    temperature_data = {'Max Temperature': None,
            'Avg Temperature': None,
            'Min Temperature': None,
            'Dew Point': None,
            'Precipitation': None,
            'Wind': None,
            'Sea Level Pressure': None
        }
    
    # Extract rows
    for row in table.find_all('tr')[1:]:
        rows = [rw.get_text(strip=True) for rw in row.find_all('td')]
        if len(rows) > 1:  # Ensure there are enough cells in the row
            label = rows[0]
            value = rows[1]
            if 'Max Temperature' in label:
                temperature_data['Max Temperature'] = value
            elif 'Avg Temperature' in label:
                temperature_data['Avg Temperature'] = value
            elif 'Min Temperature' in label:
                temperature_data['Min Temperature'] = value
            elif 'Dew Point' in label:
                temperature_data['Dew Point'] = value
            elif 'Precipitation' in label:
                temperature_data['Precipitation'] = value
            elif 'Wind' in label:
                temperature_data['Wind'] = value
            elif 'Sea Level Pressure' in label:
                temperature_data['Sea Level Pressure'] = value

        # Append the extracted data to the DataFrame
        df = pd.DataFrame({
            'County': [county],
            'Month': [month.capitalize()],
            'Year': [year],
            'Max Temperature': [temperature_data.get('Max Temperature')],
            'Avg Temperature': [temperature_data.get('Avg Temperature')],
            'Min Temperature': [temperature_data.get('Min Temperature')],
            'Dew Point': [temperature_data.get('Dew Point')],
            'Precipitation': [temperature_data.get('Precipitation')],
            'Wind': [temperature_data.get('Wind')],
            'Sea Level Pressure': [temperature_data.get('Sea Level Pressure')]
        })

        all_data = pd.concat([all_data, df], ignore_index=True)
        

In [25]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24960 entries, 0 to 24959
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   County              24960 non-null  object
 1   Month               24960 non-null  object
 2   Year                24960 non-null  int64 
 3   Max Temperature     24960 non-null  object
 4   Avg Temperature     23040 non-null  object
 5   Min Temperature     21120 non-null  object
 6   Dew Point           17280 non-null  object
 7   Precipitation       13440 non-null  object
 8   Wind                7680 non-null   object
 9   Sea Level Pressure  1920 non-null   object
dtypes: int64(1), object(9)
memory usage: 1.9+ MB


In [28]:
all_data.to_csv("climate_data")